### Optimizing Temporary Housing Deployment to Reduce U.S. Homelessness 
Nauman Sohani, Yannan Tuo, Charlie Nitschelm

In [ ]:
# Load packages
import gurobipy as gp
from gurobipy import GRB
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [ ]:
# Generate distance matrix
%run ./optimization/coc-distance.ipynb
%whos

# print(coc_distance_matrix.head())

Variable                   Type            Data/Info
----------------------------------------------------
GRB                        type            <class 'gurobipy._grb.GRB'>
Path                       type            <class 'pathlib.Path'>
R                          float           3958.8
abbreviation               str             WY
abbreviation_state_pairs   list            n=50
avg_lat                    float64         42.99953828079687
avg_lon                    float64         -107.55139588131806
centroids                  GeoSeries       0    POINT (-107.5514 42.99954)\ndtype: geometry
coc                        str             WY_500
coc_dest                   str             WY_500
coc_distance_arr           list            n=380
coc_distance_matrix        DataFrame                    AL_500      <...>n[380 rows x 380 columns]
coc_lon_lat                dict            n=380
coc_path                   WindowsPath     c:\Users\yanna\repos\15-u<...>oC_GIS_State_Shapefile_WY
c

In [6]:
# Set up Gurobi environment
env = gp.Env(empty=True)
env.setParam('OutputFlag', 0)
env.start()

# Initialize the model
m = gp.Model(env=env)

In [7]:
# High level variables + assumptions
# Total budget allocated for uplift units
TotalBudget = 130000000 # 50000 # 10000000

# Uplift unit specification assumptions
# Uplift Factory Spec 
FactoryCost = 10000

# Uplift THU Spec 
THUCost = 5000

# Uplift Shipping Spec 
THUShippingBase = 100
THUShippingPerMile = 1
# Uplift Operational Spec 
# THU_to_Factory_Max_Distance = 50

# Big M
M = 999999999999999999999

In [ ]:
# City distance data. What to use as baseline? Generate dictionary for all cities? (500-ish)
print(coc_distance_matrix.head())
# city_distance = {
#                  "SF": {"SF": 0, "Boston": 3000, "NYC": 2900}, 
#                  "Boston": {"SF": 3000, "Boston": 0, "NYC": 215},
#                  "NYC": {"SF": 2900, "Boston": 215, "NYC": 0}
#                  }



            AL_500      AL_501      AL_502      AL_503      AL_504  \
CoC                                                                  
AL_500    0.000000  203.480673   87.164712   81.774006   85.508827   
AL_501  203.480673    0.000000  257.379363  279.601537  145.056886   
AL_502   87.164712  257.379363    0.000000   51.905484  169.029477   
AL_503   81.774006  279.601537   51.905484    0.000000  166.363002   
AL_504   85.508827  145.056886  169.029477  166.363002    0.000000   

            AL_505      AL_506      AL_507      AL_508       AK_500  ...  \
CoC                                                                  ...   
AL_500   66.127834   51.206186   62.256556  163.153819  3313.591804  ...   
AL_501  264.200622  176.465797  142.425632  142.915175  3428.426654  ...   
AL_502  107.258748   82.293803  135.017153  247.912473  3226.664571  ...   
AL_503   66.428368  105.722426  142.673257  241.967259  3244.020027  ...   
AL_504  128.931653   98.999199   42.183269   78.93214

In [ ]:
# CoC Data/homeless population
# CoCPopulations = {"SF": 8300, "Boston": 6000, "NYC": 350000}
CoCPopulations = {"SF": 9000, "Boston": 6000, "NYC": 15000}
# CoCPopulations = {"SF": 5000, "Boston": 5000, "NYC": 5000}


# Number of cities
NumCities = numCoCs

# Number of factory locations available (equivalent to cities)
# todo: rename var
NumFactories = numCoCs

# ordered dict for version < 3.7; assumed same order in version >= 3.7
# indices = range(len(CoCPopulations))
# cities = CoCPopulations.keys()
# indexToCityDict = dict(zip(indices, cities))
# def indexToCity(index):
#     return indexToCityDict[index]

# print(indexToCityDict)

{0: 'SF', 1: 'Boston', 2: 'NYC'}


In [469]:
# Decision variables
# How many homes to place per city
THU = m.addVars((t for t in range(0, NumCities)), lb=0, name="THUQuantityPerCity")

# Where to place factories (also # of factories)
Factory = m.addVars((t for t in range(0, NumFactories)), vtype=GRB.BINARY, name="FactoryLocations")

# How many THUs shipped from a given factory
THUShippedFromFactory = m.addMVar((NumCities, NumFactories), name="ClosestFactory", lb=0)
# City = m.addVars(((i, j) for i in range(0, NumCities) for j in range(0, NumFactories)), lb=0, ub = 1, name="ClosestFactory")


In [ ]:
# Constraints
# Budget constraint
# sum cost of all the THUs + 
# sum cost of all the factories + 
# sum cost of transportation of THUs from all the factories = # produced in city * closest factory * distance to factory * shipping cost per mile
budgetConstr = m.addConstr((sum(THU[cityIndex]*THUCost for cityIndex in range(0, NumCities)) + 
                              FactoryCost*sum(Factory[factory] for factory in range(NumFactories)) +
                              sum(THUShippedFromFactory[(cityIndex, factory)]*coc_distance_matrix.iloc[cityIndex, factory])*THUShippingPerMile 
                              for cityIndex in range(NumCities) for factory in range(NumFactories))) <= TotalBudget, name='BudgetConstr')

# Unhoused population (allocate no more than # unhoused)
popConstr = m.addConstrs(THU[cityIndex] <= CoCPopulations[indexToCity(cityIndex)] for cityIndex in range(0, NumCities))

# sum of THUs shipped needs to equal THUs in location
THUTotal = m.addConstrs(sum(THUShippedFromFactory[cityIndex][factoryIndex] for factoryIndex in range(NumFactories)) == THU[cityIndex] for cityIndex in range(NumCities))

# enforce no THU production if factory not selected
THUProdatFactory = m.addConstrs(THUShippedFromFactory[cityIndex][factoryIndex] <= M*Factory[factoryIndex] for cityIndex in range(NumCities) for factoryIndex in range(NumFactories))

# Todo: max and min # THUs per factory

# There must be at least one factory in total
factoryConstr = m.addConstr(sum(Factory[factory] for factory in range(NumFactories)) >= 1)


In [471]:
# Objective function
# Maximize number of housing units provided for unhoused populations (ie; minimize unhoused individiuals)
m.setObjective(sum(THU[city] for city in range(0, NumCities)), gp.GRB.MAXIMIZE)


In [472]:
# Update and write the model
m.update() # Update model parameters
# m.write("uplift.lp") # Write model to file

# Solve
m.optimize()

# Check model is not infeasible
if m.status == GRB.INFEASIBLE:
    print("model is infeasible")
print("\nObjective value: ", "%.2f" % m.getAttr("ObjVal"))



Objective value:  25994.00


In [473]:
# Print solution
print("THUs:")
for i in THU:
    # if THU[i].x > 0:
        print('%s: %g' % (indexToCity(i), THU[i].x))

print("Factories:")
for i in Factory:
    # if THU[i].x > 0:
        print('%s: %g' % (indexToCity(i), Factory[i].x))

print("Shipping from factories:")
for i in range(NumCities):
    for j in range(NumFactories):
    # if THU[i].x > 0:
        print('source city: %s, factory city: %s, value: %g' % (indexToCity(i), indexToCity(j), THUShippedFromFactory[i][j].x))

THUExpense = sum(THU[cityIndex].x*THUCost for cityIndex in range(0, NumCities))
FactoryExpense = FactoryCost*sum(Factory[factory].x for factory in range(NumFactories))
shipCost = sum(THUShippedFromFactory[cityIndex][factory].x*city_distance[indexToCity(cityIndex)][indexToCity(factory)]*THUShippingPerMile for cityIndex in range(NumCities) for factory in range(NumFactories))
print("Expected max budget: " + str(TotalBudget) + " actual budget: " + str(THUExpense + FactoryExpense + shipCost))
print("THU cost: ", THUExpense)
print("Factory cost: " + str(FactoryExpense))
print("THU ship cost: " + str(shipCost))

THUs:
SF: 4994
Boston: 6000
NYC: 15000
Factories:
SF: 1
Boston: 1
NYC: 1
Shipping from factories:
source city: SF, factory city: SF, value: 4994
source city: SF, factory city: Boston, value: 0
source city: SF, factory city: NYC, value: 0
source city: Boston, factory city: SF, value: 0
source city: Boston, factory city: Boston, value: 6000
source city: Boston, factory city: NYC, value: 0
source city: NYC, factory city: SF, value: 0
source city: NYC, factory city: Boston, value: 0
source city: NYC, factory city: NYC, value: 15000
Expected max budget: 130000000 actual budget: 130000000.0
THU cost:  129970000.0
Factory cost: 30000.0
THU ship cost: 0.0


#### Analysis of optimization results
beep boop